In [2]:
!pip install pyyaml h5py  

In [3]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)


2.15.0


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [5]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model


In [6]:
# Create a basic model instance
model = create_model()

In [7]:
# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

Epoch 1/10

26/32 [=======================>......] - ETA: 0s - loss: 1.2464 - sparse_categorical_accuracy: 0.6502
Epoch 1: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 1.1408 - sparse_categorical_accuracy: 0.6760 - val_loss: 0.7029 - val_sparse_categorical_accuracy: 0.7840
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.3997 - sparse_categorical_accuracy: 0.8895
Epoch 2: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.4171 - sparse_categorical_accuracy: 0.8850 - val_loss: 0.5275 - val_sparse_categorical_accuracy: 0.8320
Epoch 3/10
31/32 [============================>.] - ETA: 0s - loss: 0.2786 - sparse_categorical_accuracy: 0.9264
Epoch 3: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.2794 - sparse_categorical_accuracy: 0.9260 - val_loss: 0.4768 - val_sparse_categorical_accuracy: 0.8540
Epoch 4/10
32/32 [========

In [9]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [10]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3573 - sparse_categorical_accuracy: 0.1180 - 131ms/epoch - 4ms/step
Untrained model, accuracy: 11.80%


In [11]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4333 - sparse_categorical_accuracy: 0.8590 - 79ms/epoch - 2ms/step
Restored model, accuracy: 85.90%


In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [12]:
batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)  # round up the number of batches to the nearest whole integer

In [13]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

In [14]:
# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_1\cp.ckpt

Epoch 10: saving model to training_1\cp.ckpt

Epoch 15: saving model to training_1\cp.ckpt

Epoch 20: saving model to training_1\cp.ckpt

Epoch 25: saving model to training_1\cp.ckpt

Epoch 30: saving model to training_1\cp.ckpt

Epoch 35: saving model to training_1\cp.ckpt

Epoch 40: saving model to training_1\cp.ckpt

Epoch 45: saving model to training_1\cp.ckpt

Epoch 50: saving model to training_1\cp.ckpt


In [15]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [16]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_1\\cp.ckpt'

In [17]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5055 - sparse_categorical_accuracy: 0.8780 - 123ms/epoch - 4ms/step
Restored model, accuracy: 87.80%


In [18]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1263 - sparse_categorical_accuracy: 0.6870
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4138 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2920 - sparse_categorical_accuracy: 0.9170
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2037 - sparse_categorical_accuracy: 0.9580
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1452 - sparse_categorical_accuracy: 0.9650


In [19]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
